<h1>Imports</h1>

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score
import pandas as pd
from nltk.corpus import stopwords
import emoji as em
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder

<h1>Import des données</h1>

In [ ]:
data_train = pd.read_xml("./data/apprentissage/train.xml")
data_train = data_train.dropna()

data_dev = pd.read_xml("./data/apprentissage/dev.xml")
data_dev = data_dev.dropna()

data_test = pd.read_xml("./data/test/test.xml")
data_test = data_test.dropna()

<h1>Prédictions en prénant en compte le sentiment lié au léxique du commentaire</h1>

In [ ]:
lexique = pd.read_csv("./data/externe/lexique_sentimental.csv")
lexique = dict(zip(lexique["word"].to_list(), lexique["polarity"].to_list()))
lexique

In [ ]:
temp = data_train
temppp = {0: -1, 1: 1}

def get_polarity(comment):
    score = 0
    found = False
    keys = [i for i in lexique.keys() if i in comment]
    if len(keys)>=1:
        found = True
        for i in keys:  
            score += temppp[lexique[i]]
        if score<0:
            return "negative"
        elif score>0:
            return "positive"
        else:
            return "neutral"
    else:
        return "not data"

temp["lexique"] = temp["commentaire"].apply(lambda x: get_polarity(x))

In [ ]:
from sklearn.preprocessing import OneHotEncoder

tfidf = TfidfVectorizer(
    stop_words = stopwords.words("french") + stopwords.words("english"),
    analyzer = 'word',
    lowercase = True,
    ngram_range = (1,2)
)
preprocessor = ColumnTransformer(transformers=[
                ('tfidf', tfidf, "commentaire"),
                ("scaler", OneHotEncoder(), ['emoticon'])
            ])
model = make_pipeline(preprocessor, SGDClassifier(loss = "hinge", penalty = "l2"))
model.fit(temp, temp["note"])

In [ ]:
y_pred = model.predict(temp2)
accuracy_score(data_dev["note"], y_pred)